# Example #2: Exploring Recent Shootings in Philadelphia with Altair, Folium, and Panel

In this notebook, we create an example dashboard visualizing recent shootings in Philadelphia. The dashboard includes:

- a Folium heatmap of the shootings from the past *X* days, where *X* is specified by the user
- an Altair histogram of victim ages
- an hvplot bar chart showing the number of shootings per day
- a slider for selecting the number of days to query shootings for

#### References
​
For more information, see the documentation:
​
- [Documentation homepage](https://panel.holoviz.org)
- [App Gallery](https://panel.holoviz.org/gallery/index.html)
    - Examples of end-to-end web apps using Panel
- [Component Gallery](https://panel.holoviz.org/reference/index.html)
    - Examples (code snippets) for the types of panes, widgets, and layouts possible in Panel dashboards
- [Folium documentation](https://python-visualization.github.io/folium/)

​
::: {.callout-note}
​
For this dashboard, check out the following reference examples:

1. See the **Altair/Vega** [reference gallery page](https://panel.holoviz.org/reference/panes/Vega.html) for more information about embedding Altair in Panel dashboards.

2. See the **Folium** [reference gallery page](https://panel.holoviz.org/reference/panes/Folium.html) for more information about embedding Folium in Panel dashboards.


:::

In [1]:
import panel as pn

pn.extension("vega")  # Enables support for vega/altair

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

import holoviews as hv
import hvplot.pandas
import param as pm

import altair as alt
import folium
from folium.plugins import HeatMap

## Load the full data set

We'll load the full data set before initializing the app, and then the user will be able to filter and display subsets of the entire data set. 

The maximum of our slider will be 365 days, so we only need to query data from the past year. When the user slides the slider, we can select the subset of this full dataset to apply.

In [3]:
# The maximum number of days into past to query
# NOTE: this will be the maximum number of days the user will be allowed to select
MAX_DAYS = 365
MIN_DAYS = 30

### Query the CARTO API

Get shooting victim data from Open Data Philly for all data within the last 365 days.

[API Documentation](https://cityofphiladelphia.github.io/carto-api-explorer/#shootings)

In [4]:
# The CARTO API endpoint
CARTO_URL = "https://phl.carto.com/api/v2/sql"

# Select everything within the last year
# NOTE: "current_date" is a SQL variable built in to the database
WHERE = f"date_ >= current_date - {MAX_DAYS}"

# The query parameters
params = {"q": "SELECT * FROM shootings", "format": "geojson", "where": WHERE}

# The request
response = requests.get(CARTO_URL, params=params)

# Make the GeoDataFrame
DATA = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")

### Preprocess the data

Do some preprocessing to clean up the data and format some columns.

In [5]:
# Remove entries with missing values
DATA = DATA.dropna(subset=["point_x", "point_y"])

# Re-map the fatal column to Yes/No
DATA["fatal"] = DATA["fatal"].map({0: "No", 1: "Yes"})

# Remove entries where fatal is NaN
DATA = DATA.dropna(subset=["fatal"])

# Parse the date_ column into a valid Datetime object
DATA["date_"] = pd.to_datetime(DATA["date_"].str.slice(0, 10), format="%Y-%m-%d")

# Add longitude and latitude columns
DATA["lat"] = DATA.geometry.y
DATA["lng"] = DATA.geometry.x

## Build the app

We'll define our app as a custom Python class that defines the various components of our dashboard, which include:

- The parameters we want the user to be able to change.
- Functions to generate the various charts/maps in our dashboard, based on those input parameters.
- The dependencies between our chart functions and parameters.

In [11]:
class PhiladelphiaShootingsApp(pm.Parameterized):
    """
    An example app visualizing recent shootings in Philadelphia.

    It includes:
    - a slider widget for selecting the number of days to query shootings for
    - a Folium heatmap
    - an Altair histogram
    - an hvplot line chart

    There is one parameter:

        - "days": the number of days to query shootings for
    """

    # the number of days to get data for
    days = pm.Integer(default=90, bounds=(MIN_DAYS, MAX_DAYS))

    def filter_by_days(self):
        """
        Return the subset of the full data set ('DATA') that
        occurred in the last 'self.days' days.
        """
        # Today's date
        today = pd.to_datetime("today")

        # Difference between shootings and today
        diff = today - DATA["date_"]

        # Valid selection: less than X days ago
        selection = diff.dt.days < self.days

        # only return subset of data that is necessary
        subset = DATA.loc[selection]

        # only return columns we need
        COLS = ["age", "fatal", "lat", "lng", "date_"]
        subset = subset[COLS]

        return subset

    def get_daily_shootings(self):
        """
        Return a DataFrame with two columns ('date_', 'count')
        that gives the total number of shootings in a day.
        """
        # data from past X days
        df = self.filter_by_days()

        # this will group our the current index by days
        # see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html
        grouper = pd.Grouper(freq="d")

        return df.set_index("date_").groupby(grouper).size().reset_index(name="count")

    @pm.depends("days")
    def pane_daily_shootings_chart(self):
        """
        Return an hvplot chart of daily shootings.

        The data displayed will only be within the currently
        selected x-axis bounds.
        """
        # get the daily shootings (filtered by days)
        N = self.get_daily_shootings()

        # create the background curve, that does not care about x-axis selection
        # This is gray!
        return N.hvplot.line(
            x="date_",
            y="count",
            color="#cfcfcf",
            width=1200,
            height=300,
        )

    @pm.depends("days")
    def pane_heatmap(self):
        """
        Return a Folium map with a heatmap showing the currently
        selected data.
        """
        # initialize the Folium map
        m = folium.Map(
            location=[39.99, -75.13], tiles="Cartodb Positron", zoom_start=12
        )

        # get the filtered data
        data = self.filter_by_days()

        # convert to coordinates array
        # Syntax is (lat, lng)
        coordinates = data[["lat", "lng"]].values

        # add heat map
        HeatMap(coordinates).add_to(m)

        # return the Pane object with a width/height
        return pn.pane.plot.Folium(m, height=600)

    @pm.depends("days")
    def pane_summary_text(self):
        """
        Get a summary of the number of shootings/homicides.

        Returns an HTML <p> tag.
        """
        # only filter this by days
        data = self.filter_by_days()

        # count shootings and homicides
        shootings = len(data)
        homicides = (data.fatal == "Yes").sum()
        t = f"<p style='font-size: 20px'>There have been {shootings:,} shootings and {homicides:,} homicides in the last {self.days} days.</p>"

        return pn.pane.HTML(t, width=550)

    @pm.depends("days")
    def pane_altair_hist(self):
        """
        Return an altair histogram of the number of currently selected
        shootings by age.
        """
        # get the filtered data
        data = self.filter_by_days()

        # create the chart
        chart = (
            alt.Chart(data)
            .mark_bar()
            .encode(
                x=alt.X("age:Q", bin=True, scale=alt.Scale(domain=(0, 100))),
                y="count()",
                color="fatal:N",
                tooltip=["count()", "age", "fatal"],
            )
            .properties(
                width=400, height=400, title="Number of Shootings by Victim's Age"
            )
        )
        return pn.pane.Vega(chart, width=600)  # NOTE: pad the width for the legend

In [12]:
app = PhiladelphiaShootingsApp(name="")

## Layout our Panel object: Bootstrap template

We can use a combination of the `Column()` and `Row()` objects in Panel to create the layout in the main component.

We'll also use the [Bootstrap template](https://panel.holoviz.org/reference/templates/Bootstrap.html) to improve the aesthetics of the user interface.

::: {.callout-note}

- The `app.param` is an automatically generated set of widgets that corresponds to our `param` parameters
- The charts are specified as the functions of our main application, e.g., app.bar_chart is the function that will return our bar chart.
:::

Create a text HTML element to show instructions below the daily shootings chart and the title of the app:

In [13]:
panel = pn.template.BootstrapTemplate(
    title="Recent Shootings in Philadelphia",
    sidebar_width=0,  # No sidebar needed for this app
)

In [14]:
# Append the content to the main part of the dashboard
panel.main.append(
    pn.Column(
        pn.Row(app.pane_summary_text, app.param),
        pn.Row(app.pane_altair_hist, app.pane_heatmap),
        pn.Row(app.pane_daily_shootings_chart, align="center"),
    )
)

### Call servable() and render our Panel object

- The final step is to call the `.servable()` function
- This will render the dashboard directly in the notebook
- It also enables the notebook to be served from `localhost`.

::: {.callout-warning}
Since we're using a template, the dashboard won't render properly in the notebook! You'll need to run it from the command line (Miniforge Prompt or Terminal).
:::


From the command line, we will run:

```
panel serve --show app2.ipynb
```

And see the app live at: `http://localhost:5006/app2`

In [15]:
panel.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo(dark_mode=False, device_pixel_ratio=2, language='en-US', timezone='America/New_York', timezone_offset=300, webdriver=False)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-6305175760] Column
        [0] Row
            [0] ParamMethod(method, _pane=HTML, defer_load=False)
            [1] Column(margin=(5, 10))
                [0] StaticText(value='<b></b>')
                [1] IntSlider(end=365, name='Days', start=30, value=90)
        [1] Row
            [0] ParamMethod(method, _pane=Vega, defer_load=False)
            [1] ParamMethod(method, _pane=Folium, defer_load=False)
        [2] Row(align='center')
            [0] ParamMethod(method, _pane=HoloViews, defer_load=False)